# 🎯 v17b: CV-LB Gap Reduction Inference

**목표**: CV-LB Gap 최소화 (0.10 → 0.05 이하)

**전략**:
1. 7x TTA (Original + Flips + Rotations)
2. 예측값 Clipping (outlier 제거)
3. Ensemble 평균 대신 Median 사용 옵션

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['PYTHONWARNINGS'] = 'ignore'

import gc
import random
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image
from tqdm.auto import tqdm
from scipy import stats

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
import torchvision.transforms.functional as TF

import timm

tqdm.pandas()

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

seed_everything(42)

## ⚙️ Configuration

In [ ]:
class CFG:
    DATA_PATH = Path("/kaggle/input/csiro-biomass")
    
    # ⚠️ 이 경로를 업로드한 모델 Dataset 경로로 변경하세요
    MODELS_DIR = Path("/kaggle/input/csiro-v17-models")
    
    # 표준 ViT-Large 아키텍처 (timm이 인식 가능)
    # 실제 weights는 fold model에서 로드됨
    model_name = "vit_large_patch16_224"
    img_size = (512, 512)
    
    # v17과 동일한 head 구조
    hidden_dim = 512
    num_layers = 3
    dropout = 0.1
    use_layernorm = True
    
    # === CV-LB Gap 감소 전략 ===
    use_strong_tta = True      # 7x TTA
    use_clipping = True        # Prediction clipping
    use_median = False         # Median ensemble (alternative)
    
    # Clipping bounds (train 데이터 기반)
    clip_bounds = {
        'Dry_Green_g': (0, 150),
        'Dry_Dead_g': (0, 150),
        'Dry_Clover_g': (0, 80),
        'GDM_g': (0, 200),
        'Dry_Total_g': (0, 250),
    }
    
    batch_size = 32
    num_workers = 0
    device = "cuda" if torch.cuda.is_available() else "cpu"

cfg = CFG()
print(f"Device: {cfg.device}")
print(f"Strong TTA: {cfg.use_strong_tta}")
print(f"Clipping: {cfg.use_clipping}")

## 📊 Dataset

In [ ]:
class TestDataset(Dataset):
    def __init__(self, df, cfg, transform=None):
        self.df = df.reset_index(drop=True)
        self.cfg = cfg
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(self.cfg.DATA_PATH / row['image_path']).convert('RGB')
        width, height = img.size
        mid = width // 2
        
        left_img = img.crop((0, 0, mid, height))
        right_img = img.crop((mid, 0, width, height))
        
        if self.transform:
            left_img = self.transform(left_img)
            right_img = self.transform(right_img)
        
        return left_img, right_img, row['sample_id_prefix']

def get_tta_dataloaders(df, cfg):
    """7x TTA: Original + Flips + Rotations"""
    loaders = []
    
    base_transform = [
        T.Resize(cfg.img_size),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]
    
    if cfg.use_strong_tta:
        # 7x TTA
        transforms_list = [
            # 1. Original
            T.Compose(base_transform),
            # 2. Horizontal Flip
            T.Compose([T.Resize(cfg.img_size), T.RandomHorizontalFlip(p=1.0)] + base_transform[1:]),
            # 3. Vertical Flip
            T.Compose([T.Resize(cfg.img_size), T.RandomVerticalFlip(p=1.0)] + base_transform[1:]),
            # 4. Rotate 90
            T.Compose([T.Resize(cfg.img_size), T.Lambda(lambda x: TF.rotate(x, 90))] + base_transform[1:]),
            # 5. Rotate 180
            T.Compose([T.Resize(cfg.img_size), T.Lambda(lambda x: TF.rotate(x, 180))] + base_transform[1:]),
            # 6. Rotate 270
            T.Compose([T.Resize(cfg.img_size), T.Lambda(lambda x: TF.rotate(x, 270))] + base_transform[1:]),
            # 7. HFlip + Rotate 90
            T.Compose([T.Resize(cfg.img_size), T.RandomHorizontalFlip(p=1.0), 
                      T.Lambda(lambda x: TF.rotate(x, 90))] + base_transform[1:]),
        ]
        print(f"Using 7x TTA")
    else:
        # 3x TTA
        transforms_list = [
            T.Compose(base_transform),
            T.Compose([T.Resize(cfg.img_size), T.RandomHorizontalFlip(p=1.0)] + base_transform[1:]),
            T.Compose([T.Resize(cfg.img_size), T.RandomVerticalFlip(p=1.0)] + base_transform[1:]),
        ]
        print(f"Using 3x TTA")
    
    for transform in transforms_list:
        dataset = TestDataset(df, cfg, transform)
        loader = DataLoader(dataset, batch_size=cfg.batch_size,
                           shuffle=False, num_workers=cfg.num_workers, pin_memory=True)
        loaders.append(loader)
    
    return loaders

## 🧠 Model

In [ ]:
class FiLM(nn.Module):
    def __init__(self, feat_dim):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(feat_dim, feat_dim // 2),
            nn.ReLU(inplace=True),
            nn.Linear(feat_dim // 2, feat_dim * 2)
        )
    
    def forward(self, context):
        gamma_beta = self.mlp(context)
        return torch.chunk(gamma_beta, 2, dim=1)


def make_head(in_dim: int, hidden_dim: int, num_layers: int, dropout: float, use_layernorm: bool):
    layers = []
    current_dim = in_dim
    
    for i in range(num_layers):
        out_dim = hidden_dim if i < num_layers - 1 else 1
        layers.append(nn.Linear(current_dim, out_dim if i < num_layers - 1 else hidden_dim))
        
        if i < num_layers - 1:
            if use_layernorm:
                layers.append(nn.LayerNorm(hidden_dim))
            layers.append(nn.ReLU(inplace=True))
            layers.append(nn.Dropout(dropout))
        current_dim = hidden_dim
    
    layers.append(nn.Linear(hidden_dim, 1))
    return nn.Sequential(*layers)


class CSIROModelV17(nn.Module):
    """v17 모델 - 표준 ViT 아키텍처로 시작, fold weights로 덮어씀"""
    def __init__(self, cfg):
        super().__init__()
        
        # 표준 ViT-Large 아키텍처 (timm이 인식 가능)
        # pretrained=False로 빈 모델 생성, 나중에 fold weights로 채움
        self.backbone = timm.create_model(cfg.model_name, pretrained=False, num_classes=0, global_pool='avg')
        
        feat_dim = self.backbone.num_features  # 1024
        combined_dim = feat_dim * 2
        
        self.film = FiLM(feat_dim)
        
        self.head_green = make_head(combined_dim, cfg.hidden_dim, cfg.num_layers, cfg.dropout, cfg.use_layernorm)
        self.head_clover = make_head(combined_dim, cfg.hidden_dim, cfg.num_layers, cfg.dropout, cfg.use_layernorm)
        self.head_dead = make_head(combined_dim, cfg.hidden_dim, cfg.num_layers, cfg.dropout, cfg.use_layernorm)
        
        self.softplus = nn.Softplus(beta=1.0)
    
    def forward(self, left_img, right_img):
        left_feat = self.backbone(left_img)
        right_feat = self.backbone(right_img)
        
        context = (left_feat + right_feat) / 2
        gamma, beta = self.film(context)
        
        left_mod = left_feat * (1 + gamma) + beta
        right_mod = right_feat * (1 + gamma) + beta
        
        combined = torch.cat([left_mod, right_mod], dim=1)
        
        green = self.softplus(self.head_green(combined))
        clover = self.softplus(self.head_clover(combined))
        dead = self.softplus(self.head_dead(combined))
        
        gdm = green + clover
        total = gdm + dead
        
        return torch.cat([green, dead, clover, gdm, total], dim=1)

## 🔮 Inference with Gap Reduction

In [ ]:
@torch.no_grad()
def predict(model, loader, device):
    model.eval()
    all_outputs, all_ids = [], []
    
    for left, right, ids in tqdm(loader, desc="Predicting"):
        left = left.to(device)
        right = right.to(device)
        outputs = model(left, right)
        all_outputs.append(outputs.cpu().numpy())
        all_ids.extend(ids)
    
    return np.concatenate(all_outputs), all_ids


def predict_with_tta(model, tta_loaders, device, use_median=False):
    all_preds = []
    final_ids = None
    
    for loader in tta_loaders:
        preds, ids = predict(model, loader, device)
        all_preds.append(preds)
        if final_ids is None:
            final_ids = ids
    
    if use_median:
        # Median은 outlier에 더 robust
        avg_preds = np.median(all_preds, axis=0)
    else:
        avg_preds = np.mean(all_preds, axis=0)
    
    return avg_preds, final_ids


def clip_predictions(preds, cfg):
    """Outlier 예측값 clipping"""
    TARGET_ORDER = ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']
    
    clipped = preds.copy()
    for i, target in enumerate(TARGET_ORDER):
        low, high = cfg.clip_bounds[target]
        original_min, original_max = clipped[:, i].min(), clipped[:, i].max()
        clipped[:, i] = np.clip(clipped[:, i], low, high)
        new_min, new_max = clipped[:, i].min(), clipped[:, i].max()
        if original_max > high or original_min < low:
            print(f"  {target}: [{original_min:.1f}, {original_max:.1f}] → [{new_min:.1f}, {new_max:.1f}]")
    
    return clipped


def predict_ensemble(cfg, tta_loaders):
    all_fold_preds = []
    final_ids = None
    
    model_files = sorted(cfg.MODELS_DIR.glob("model_fold*.pth"))
    print(f"Found {len(model_files)} models")
    
    for model_file in model_files:
        print(f"\nLoading {model_file.name}...")
        
        # 1. 빈 모델 아키텍처 생성
        model = CSIROModelV17(cfg).to(cfg.device)
        
        # 2. fold weights 로드 (backbone 포함 전체 덮어씀)
        state_dict = torch.load(model_file, map_location=cfg.device)
        model.load_state_dict(state_dict)
        print("✓ Weights loaded")
        
        preds, ids = predict_with_tta(model, tta_loaders, cfg.device, cfg.use_median)
        all_fold_preds.append(preds)
        
        if final_ids is None:
            final_ids = ids
        
        del model
        gc.collect()
        torch.cuda.empty_cache()
    
    # Fold ensemble
    if cfg.use_median:
        final_preds = np.median(all_fold_preds, axis=0)
    else:
        final_preds = np.mean(all_fold_preds, axis=0)
    
    # Clipping
    if cfg.use_clipping:
        print("\n📏 Applying prediction clipping...")
        final_preds = clip_predictions(final_preds, cfg)
    
    return final_preds, final_ids

## 📋 Main

In [ ]:
test_df = pd.read_csv(cfg.DATA_PATH / "test.csv")
test_df['sample_id_prefix'] = test_df['sample_id'].str.split('__').str[0]
test_wide = test_df.drop_duplicates(subset=['image_path']).reset_index(drop=True)
print(f"Test samples: {len(test_wide)}")

In [ ]:
tta_loaders = get_tta_dataloaders(test_wide, cfg)
preds, sample_ids = predict_ensemble(cfg, tta_loaders)
print(f"\nPredictions: {preds.shape}")

In [ ]:
TARGET_ORDER = ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']

pred_df = pd.DataFrame(preds, columns=TARGET_ORDER)
pred_df['sample_id_prefix'] = sample_ids

sub_df = pred_df.melt(
    id_vars=['sample_id_prefix'],
    value_vars=TARGET_ORDER,
    var_name='target_name',
    value_name='target'
)
sub_df['sample_id'] = sub_df['sample_id_prefix'] + '__' + sub_df['target_name']

submission = sub_df[['sample_id', 'target']]
submission.to_csv('submission.csv', index=False)

print(f"\n✅ Saved: {len(submission)} rows")
print("\nPrediction statistics:")
for col in TARGET_ORDER:
    vals = pred_df[col]
    print(f"  {col}: mean={vals.mean():.2f}, std={vals.std():.2f}, min={vals.min():.2f}, max={vals.max():.2f}")

In [ ]:
sample_sub = pd.read_csv(cfg.DATA_PATH / "sample_submission.csv")
assert len(submission) == len(sample_sub), "Row count mismatch!"
print("\n✓ Format verified!")